In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, ElasticNet

df = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")
y = df["type"]
y_train_nn = pd.get_dummies(df[["type"]], prefix="")
df = df.drop("type",axis=1)
#df.drop("color", inplace=True, axis=1)
#df_test.drop("color", inplace=True, axis=1)
df = pd.get_dummies(df)
df_test = pd.get_dummies(df_test)

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df['bone_rotting'] = df['bone_length']*df['rotting_flesh']
df['bone_soul'] = df['bone_length']*df['has_soul']
df['bone_hair'] = df['bone_length']*df['hair_length']
df['flesh_hair'] = df['rotting_flesh']*df['hair_length']
df['flesh_soul'] = df['rotting_flesh']*df['has_soul']


df_test['bone_rotting'] = df_test['bone_length']*df_test['rotting_flesh']
df_test['bone_soul'] = df_test['bone_length']*df_test['has_soul']
df_test['bone_hair'] = df_test['bone_length']*df_test['hair_length']
df_test['flesh_hair'] = df_test['rotting_flesh']*df_test['hair_length']
df_test['flesh_soul'] = df_test['rotting_flesh']*df_test['has_soul']

#df = df[["bone_length","rotting_flesh","hair_length","color","has_soul"]]
#df_test = [["bone_length","rotting_flesh","hair_length","color","has_soul"]]
#df_test.fillna(0,inplace=True)
#df.fillna(0,inplace=True)

df_test.replace([np.inf, -np.inf], 100,inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)

rfc = RandomForestClassifier(n_estimators=750,max_depth=7,n_jobs=-1)
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test) 

print(accuracy_score(y_pred,y_test))

0.8


In [3]:
from sklearn.neural_network import MLPClassifier

NN = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(15,3),verbose=True,random_state =42)
NN.fit(X_train,y_train)
y_pred = NN.predict(X_test) 

print(accuracy_score(y_pred,y_test))

0.306666666667


In [4]:
import logging
logging.getLogger().setLevel(logging.INFO)

from sklearn import datasets, svm, cross_validation, tree, preprocessing, metrics
import sklearn.ensemble as ske
import tensorflow as tf
from tensorflow.contrib import learn
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.preprocessing import LabelEncoder as LE

letype = LE()
y_NN = letype.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(df, y_NN, test_size=0.2, random_state=0)


classifier = learn.DNNClassifier(hidden_units=[15], n_classes=3)
classifier.fit(x=X_train, y=y_train, max_steps=5500)
predictions = classifier.predict(X_train)

print(accuracy_score(y_train, predictions))

Instructions for updating:
Pass `tf.contrib.learn.infer_real_valued_columns_from_input(x)` or `tf.contrib.learn.infer_real_valued_columns_from_input_fn(input_fn)` as `feature_columns`, where `x` or `input_fn` is your argument to `fit`, `evaluate`, or `predict`.
INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=16, default_value=None, dtype=tf.float32)
INFO:tensorflow:Create CheckpointSaver
INFO:tensorflow:Step 1: loss = 64.5151
INFO:tensorflow:Step 101: loss = 1.17137
INFO:tensorflow:Step 201: loss = 1.5902
INFO:tensorflow:Saving checkpoints for 300 into /tmp/tmpmsm3qydi/model.ckpt.
INFO:tensorflow:Step 301: loss = 1.6059
INFO:tensorflow:Step 401: loss = 1.27795
INFO:tensorflow:Step 501: loss = 1.22352
INFO:tensorflow:Saving checkpoints for 600 into /tmp/tmpmsm3qydi/model.ckpt.
INFO:tensorflow:Step 601: loss = 1.18336
INFO:tensorflow:Step 701: loss = 1.139
INFO:tensorflow:Step 801: loss = 1.08142
INFO:tensorflow:Saving checkpoints for 900 into /tmp

0.726351351351


In [5]:
X_test = df_test
pred = classifier.predict(X_test)
pred = letype.inverse_transform(pred)
pred
output = pd.DataFrame({"id": df_test["id"],"type":pred})
output.to_csv('DNN_Classifier.csv',index=False)

INFO:tensorflow:Transforming feature_column _RealValuedColumn(column_name='', dimension=16, default_value=None, dtype=tf.float32)
INFO:tensorflow:Loading model from checkpoint: /tmp/tmpmsm3qydi/model.ckpt-5500-?????-of-00001.


In [6]:
## A dead simple neural network class in Python+Numpy. Plain SGD, and no regularization.
def sigmoid(X):
    return 1.0 / ( 1.0 + np.exp(-X) )

def softmax(X):
    _sum = np.exp(X).sum()
    return np.exp(X) / _sum

class neuralnet(object):
    def __init__(self, num_input, num_hidden, num_output):
        self._W1 = (np.random.random_sample((num_input, num_hidden)) - 0.5).astype(np.float32)
        self._b1 = np.zeros((1, num_hidden)).astype(np.float32)
        self._W2 = (np.random.random_sample((num_hidden, num_output)) - 0.5).astype(np.float32)
        self._b2 = np.zeros((1, num_output)).astype(np.float32)

    def forward(self,X):
        net1 = np.matmul( X, self._W1 ) + self._b1
        y = sigmoid(net1)
        net2 = np.matmul( y, self._W2 ) + self._b2
        z = softmax(net2)
        return z,y

    def backpropagation(self, X, target, eta):
        z, y = self.forward(X)
        d2 = (z - target)
        d1 = y*(1.0-y) * np.matmul(d2, self._W2.T)
        # The updates are done within this method. This more or less implies
        # utpdates with Stochastic Gradient Decent. Let's fix that later.
        # TODO: Support for full batch and mini-batches etc.
        self._W2 -= eta * np.matmul(y.T,d2)
        self._W1 -= eta * np.matmul(X.reshape((-1,1)),d1)
        self._b2 -= eta * d2
        self._b1 -= eta * d1

In [7]:
# Some hyper-parameters to tune.
num_hidden = 12
n_epochs   = 1500
eta        = 0.01
nn = neuralnet( df.shape[1], num_hidden,  y_train_nn.shape[1])

In [8]:
# (EDIT: It's much faster to convert the dataframes to numpy arrays and then iterate)
X = np.array(df, dtype=np.float32)
Y = np.array(y_train_nn, dtype=np.float32)
for epoch in range(n_epochs):
    for monster, target in zip(X,Y):
        nn.backpropagation( monster, target, eta)

In [9]:
with open('submission-{}-hidden.csv'.format(num_hidden), 'w') as f:
    f.write("id,type\n")
    for index, monster in df_test.iterrows():
        probs = nn.forward( np.array(monster, dtype=np.float32))[0]
        f.write("{},{}\n".format(df_test["id"][index], y_train_nn.columns.values[np.argmax(probs)][1:]))


In [10]:
def score(clf, train_np, random_state, folds, target, length):
    kf = KFold(n = length , n_folds=folds, shuffle = False, random_state = random_state)
    for itrain, itest in kf:
        Xtr, Xte = train_np[itrain], train_np[itest]
        ytr, yte = target[itrain], target[itest]
        clf.fit(Xtr, ytr.ravel())
        pred = pd.DataFrame(clf.predict(Xte))
        return accuracy_score(yte, pred)
    return accuracy_score(y, pred)

In [11]:
df_array = np.array(df)

In [12]:
kf = KFold(n = 371 , n_folds=3, shuffle = False, random_state = 42)
clf = ExtraTreesClassifier(n_estimators=750,max_depth=7,n_jobs=-1,random_state=42)
#clf = LogisticRegression(penalty='l2',C=1000000)
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,), random_state=1)

i = 0
for itrain, itest in kf:
    print(i)
    Xtr, Xte = df_array[itrain], df_array[itest]
    ytr, yte = y[itrain], y[itest]
    clf.fit(Xtr, ytr.ravel())
    if i == 0:
        pred_0 = pd.DataFrame(clf.predict(df_test))
        pred_0.columns = ['0']
    if i == 1:
        pred_1 = pd.DataFrame(clf.predict(df_test))
        pred_1.columns = ['1']
    if i == 2:
        pred_2 = pd.DataFrame(clf.predict(df_test))  
        pred_2.columns = ['2']
    i = i + 1  

clf.fit(df, y)
pred_tot = clf.predict(df_test)
Y = pd.DataFrame()
Y["id"] = df_test["id"]
Y["type"] = pred_tot
Y.to_csv("submission_total.csv",index=False)

Y = pd.DataFrame()
Y["id"] = df_test["id"]
Y["type"] = pred_0
Y.to_csv("submission0.csv",index=False)

Y = pd.DataFrame()
Y["id"] = df_test["id"]
Y["type"] = pred_1
Y.to_csv("submission1.csv",index=False)

Y = pd.DataFrame()
Y["id"] = df_test["id"]
Y["type"] = pred_2
Y.to_csv("submission2.csv",index=False)

result = pd.concat([pred_0, pred_1, pred_2], axis=1)

result['Ghoul0']=(result['0']=='Ghoul').astype(int)
result['Ghoul1']=(result['1']=='Ghoul').astype(int)
result['Ghoul2']=(result['2']=='Ghoul').astype(int)

result['Ghost0']=(result['0']=='Ghost').astype(int)
result['Ghost1']=(result['1']=='Ghost').astype(int)
result['Ghost2']=(result['2']=='Ghost').astype(int)

result['Gobelin0']=(result['0']=='Goblin').astype(int)
result['Gobelin1']=(result['1']=='Goblin').astype(int)
result['Gobelin2']=(result['2']=='Goblin').astype(int)

result['Ghoul_fin'] = result['Ghoul0'] + result['Ghoul1'] + result['Ghoul2']
result['Ghost_fin'] = result['Ghost0'] + result['Ghost1'] + result['Ghost2']
result['Gobelin_fin'] = result['Gobelin0'] + result['Gobelin1'] + result['Gobelin2']

result['type'] = 'Ghoul'

for i in range(1,529):
    if result['Ghoul_fin'][i]>1:
        result['type'][i] = 'Ghoul'
    if result['Ghost_fin'][i]>1:
        result['type'][i] = 'Ghost'
    if result['Gobelin_fin'][i]>1:
        result['type'][i] = 'Goblin'     

Y = pd.DataFrame()
Y["id"] = df_test["id"]
Y["type"] = result['type']
Y.to_csv("submission_fin.csv",index=False)        

0
1
2


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
